Подготовка модели

In [1]:
%%bash
pip install numpy scipy scikit-image matplotlib

In [2]:
pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215250 sha256=8cd268a6494e4fb266c8fe0fb6b233f3f6851df6e6ea1957d87e8188ce5da273
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pyTelegramBotAPI


In [3]:
pip install gtts

In [4]:
pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 39.2 MB/s eta 0:00:00


In [5]:
pip install pydub

In [6]:
pip install pydub[ffmpeg]

In [7]:
import torch
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn

In [8]:
pip install pydub opencv-python-headless opuslib


  Preparing metadata (setup.py) ... done
  Created wheel for opuslib: filename=opuslib-3.0.1-py3-none-any.whl size=12565 sha256=541ce68dde58de6b724c7c42c9254e14d113ac1aa50de686a22a708fabea970b
  Stored in directory: /root/.cache/pip/wheels/94/54/3c/4cdec9effae0b0ed861a7623a7d2e2a9aa2d2cad179b9d71ea
Successfully built opuslib


In [9]:
ssd_model.to('cuda')
ssd_model.eval()

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [10]:
from google.colab import drive
drive.mount('/content/drive')
torch.save(ssd_model.state_dict(), 'ssd_model.pth')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def filter2(uri):
    is_detected_cat = 0
    inputs = [utils.prepare_input(uri)]
    tensor = utils.prepare_tensor(inputs, precision == 'fp16')

    with torch.no_grad():
        detections_batch = ssd_model(tensor)

    results_per_input = utils.decode_results(detections_batch)
    best_results_per_input = [utils.pick_best(results, 0.20) for results in results_per_input]
    classes_to_labels = utils.get_coco_object_dictionary()

    for image_idx in range(len(best_results_per_input)):
        fig, ax = plt.subplots(1)
        # Show original, denormalized image...
        image = inputs[image_idx] / 2 + 0.5
        ax.imshow(image)
        # ...with detections
        bboxes, classes, confidences = best_results_per_input[image_idx]
        for idx in range(len(bboxes)):
            left, bot, right, top = bboxes[idx]
            x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            class_cat = classes_to_labels[classes[idx] - 1]
            if (class_cat == 'cat'):
                is_detected_cat = 1
            ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    #plt.show()
    plt.savefig("filtered_image.jpg")
    return is_detected_cat

Бот

In [12]:
import telebot
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.patches as patches

import telebot
import os
from gtts import gTTS
import speech_recognition as sr

from pydub import AudioSegment
import subprocess

import time

In [18]:
# Токен вашего бота, полученный от BotFather в Телеграм
#bot_token = "959861657:AAEh0AQ8AcIwVrU_x_TqbsNeNPGKAUx8LDY"
bot_token = "6707575753:AAHmJZto5V1UDNLEcLaOZzupLpZYGypJRd4"
# Создаем экземпляр бота
bot = telebot.TeleBot(bot_token)

In [19]:

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет! Пожалуйста, загрузи картинку.')


In [20]:
@bot.message_handler(content_types=['photo'])
def process_image(message):
    # Получаем информацию о картинке
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    file_path = file_info.file_path

    # Скачиваем картинку
    downloaded_file = bot.download_file(file_path)
    # Сохраняем картинку на диск
    with open('image.jpg', 'wb') as f:
        f.write(downloaded_file)

    is_detected_cat = filter2("image.jpg")

    # Отправляем пользователю преобразованное изображение
    with open('filtered_image.jpg', 'rb') as f:
        bot.send_photo(message.chat.id, f)

    if is_detected_cat == True:
        bot.reply_to(message, "Пришел твой кот. Кормушка открыта")



In [21]:

@bot.message_handler(content_types=['voice'])
def process_voice_message(message):
    # Получаем информацию о голосовом сообщении
    file_id = message.voice.file_id
    file_info = bot.get_file(file_id)
    file_path = file_info.file_path

    # Скачиваем голосовое сообщение
    downloaded_file = bot.download_file(file_path)

    # Сохраняем голосовое сообщение на диск
    voice_file_path = 'voice_message.ogg'
    with open(voice_file_path, 'wb') as f:
        f.write(downloaded_file)

    # Преобразование голосового сообщения в текст
    input_file = 'voice_message.ogg'
    audio = AudioSegment.from_file(input_file, format='ogg')

    # Конвертация в формат WAV
    output_file = 'audio.wav'
    audio.export(output_file, format='wav')


    recognizer = sr.Recognizer()
    with sr.AudioFile(output_file) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data, language='ru')  # Здесь можно указать другой язык, если нужно
        print(text)


    #text = "ваше голосовое обрабатывается"
    # Отправляем текст пользователю
    bot.reply_to(message, text)

    # Опционально: Создаем аудиофайл с ответом и отправляем его пользователю
    #tts = gTTS(text=text, lang='ru')  # Здесь можно указать другой язык, если нужно
    #tts.save('response_audio.mp3')
    #with open('response_audio.mp3', 'rb') as f:
    #    bot.send_voice(message.chat.id, f)

    # Удаляем временные файлы
    os.remove(output_file)
    #os.remove(downloaded_file)



In [ ]:
bot.polling()

In [ ]:
import sys
print(sys.version)